In [5]:
import requests
from datetime import datetime
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
import pandas as pd
import os

In [2]:
result = requests.get('https://api.binance.com/api/v3/ticker/price')
js = result.json()
symbols = [x['symbol'] for x in js]
symbols_usdt = [x for x in symbols if 'USDT' in x]  # 끝이 USDT로 끝나는 심볼들, ['BTCUSDT', 'ETHUSDT', ...]

In [4]:
COLUMNS = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'quote_av', 'trades', 
                   'tb_base_av', 'tb_quote_av', 'ignore']
URL = 'https://api.binance.com/api/v3/klines'
def get_data(start_date, end_date, symbol):
    data = []
    
    start = int(time.mktime(datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000
    end = int(time.mktime(datetime.strptime(end_date +' 23:59', '%Y-%m-%d %H:%M').timetuple())) * 1000
    params = {
        'symbol': symbol,
        'interval': '1s',
        'limit': 1000,
        'startTime': start,
        'endTime': end
    }
    
    while start < end:
        print(datetime.fromtimestamp(start // 1000))
        params['startTime'] = start
        result = requests.get(URL, params = params)
        js = result.json()
        if not js:
            break
        data.extend(js)  # result에 저장
        start = js[-1][0] + 60000  # 다음 step으로
    # 전처리
    if not data:  # 해당 기간에 데이터가 없는 경우
        print('해당 기간에 일치하는 데이터가 없습니다.')
        return -1
    
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['Open_time'] = df.apply(lambda x:datetime.fromtimestamp(x['Open_time'] // 1000), axis=1)
    df = df.drop(columns = ['Close_time', 'ignore'])
    df['Symbol'] = symbol
    df.loc[:, 'Open':'tb_quote_av'] = df.loc[:, 'Open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    
     # 데이터를 CSV 파일로 저장
    output_folder = './education/coin/jh_-/'  # 여기에 원하는 폴더 경로를 지정합니다.
    binance_csv = os.path.join(output_folder, f'{symbol}_data_{start_date}_{end_date}.csv')
    df.to_csv(binance_csv, index=False)
    print(f'데이터가 {binance_csv}에 저장되었습니다.')

    return df

start_date = '2024-01-13'
end_date = '2024-01-13'
symbol = symbols_usdt[0]
get_data(start_date, end_date, symbol)

2024-01-13 00:00:00
2024-01-13 00:17:39
2024-01-13 00:35:18
2024-01-13 00:52:57
2024-01-13 01:10:36
2024-01-13 01:28:15
2024-01-13 01:45:54
2024-01-13 02:03:33
2024-01-13 02:21:12
2024-01-13 02:38:51
2024-01-13 02:56:30
2024-01-13 03:14:09
2024-01-13 03:31:48
2024-01-13 03:49:27
2024-01-13 04:07:06
2024-01-13 04:24:45
2024-01-13 04:42:24
2024-01-13 05:00:03
2024-01-13 05:17:42
2024-01-13 05:35:21
2024-01-13 05:53:00
2024-01-13 06:10:39
2024-01-13 06:28:18
2024-01-13 06:45:57
2024-01-13 07:03:36
2024-01-13 07:21:15
2024-01-13 07:38:54
2024-01-13 07:56:33
2024-01-13 08:14:12
2024-01-13 08:31:51
2024-01-13 08:49:30
2024-01-13 09:07:09
2024-01-13 09:24:48
2024-01-13 09:42:27
2024-01-13 10:00:06
2024-01-13 10:17:45
2024-01-13 10:35:24
2024-01-13 10:53:03
2024-01-13 11:10:42
2024-01-13 11:28:21
2024-01-13 11:46:00
2024-01-13 12:03:39
2024-01-13 12:21:18
2024-01-13 12:38:57
2024-01-13 12:56:36
2024-01-13 13:14:15
2024-01-13 13:31:54
2024-01-13 13:49:33
2024-01-13 14:07:12
2024-01-13 14:24:51


NameError: name 'os' is not defined

In [ ]:
import time
years = list(range(2017, 2022))  # 바이낸스에서는 2017년 8월 이후의 데이터부터 제공
for symbol in symbols_usdt[:10]:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        df = get_data(start_date, end_date, symbol)
        df.to_csv(f'E:/projects/binance/data/{symbol[:3].lower()}_{year}.csv', index=False)  # csv파일로 저장하는 부분
				time.sleep(1)  # 과다한 요청으로 API사용이 제한되는것을 막기 위해